# Environment setups

> Run a Large Language Model using the [HuggingFace `Transformers`](https://huggingface.co/docs/transformers/index) API.  

In [1]:
#| default_exp lesson_1.first_run

The cells below are good defaults for development.  

The `autoreload` lines help load libraries on the fly, while they are changing. This works well with the editable install we created via `pip install -e .`  
This means we can edit the source code directly and have the change reflected live in the notebook.  

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Introduction

Imagine we have a list of product review from our users. Now we want to find out whether those reviews were good or bad. It will take a lot of effort to manually go through and check each one. But, using an LLM, we can automatically get a label for a given product review. 

How would this be useful? We could use it to find the more negative reviews to see where our product needs improving. Or, we can look at the more positive ones to see what we're doing right.  

The broader task in NLP of figuring out a statement's tone is called `Sentiment Analysis`.

## First, a Pipeline

A HuggingFace model is based on 3 key pieces: 
1. Config file.  
2. Preprocessor file.   
3. Model file.   

The HuggingFace API gives us a way of automatically using these pieces directly: the `pipeline`.  

Let's get right it and create a Sentiment Analysis `pipeline`.

In [3]:
#| export 

# load in the pipeline object from huggingface
from transformers import pipeline

# create the sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


We can see in the output message above that HuggingFace automatically picked a decent, default model for us since we didn't specify one. Specifically, it chose a [distilbert model](distilbert-base-uncased-finetuned-sst-2-english).  

We will learn more about what exactly `distilbert` is and how it works later on. For now, think of it as a useful NLP genie who can tell us how it feels about a given sentence. 

In [130]:
#| export

# example from the HuggingFace tutorial
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [23]:
#| export

# passing in several sentences at once, inside a python list
results = classifier([
    "We are very happy to show you the 🤗 Transformers library.",
    "We hope you don't hate it.",
    "I love Fractal! I'm so glad it's not a cult!", 
])

# print the output of each results
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309
label: POSITIVE, with score: 0.999


# Inspecting the `classifier`, notebook style.

What is the `classifier`, exactly?

In [135]:
classifier

In [ ]:
## showing the lookup's auto-complete
# classifier.

In [134]:
## viewing all of a class' methods and properties
# dir(classifier)

['__abstractmethods__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_batch_size',
 '_ensure_tensor_on_device',
 '_forward',
 '_forward_params',
 '_num_workers',
 '_postprocess_params',
 '_preprocess_params',
 '_sanitize_parameters',
 'binary_output',
 'call_count',
 'check_model_type',
 'default_input_names',
 'device',
 'device_placement',
 'ensure_tensor_on_device',
 'feature_extractor',
 'forward',
 'framework',
 'function_to_apply',
 'get_inference_context',
 'get_iterator',
 'image_processor',
 'iterate',
 'model',
 'modelcard',
 'postprocess',
 'predict',
 'preprocess',
 'return_all_scores',
 'run_mul

Jupyter notebooks have powerful ways of inspecting and analyzing the code, as we're running it. 

In [ ]:
## refresher
# classifier

In [ ]:
## the power of asking questions
# classifier?

In [ ]:
## again, with feeling
# classifier??

# Peeking inside the `pipeline`

We can see the pipeline loaded the model: ``.  

It then handled the three key pieces (Config, Preprocess, Model) underneath the hood. What exactly is `pipeline` doing?  

Let's build or own pipeline from scratch, stepping one small level below the abstraction. To do this, we will create each of the key pieces manually.  

### Config class

In [92]:
from transformers import DistilBertConfig

### Preprocessor class

In [93]:
from transformers import DistilBertTokenizer

### Model class

In [100]:
# from transformers import DistilBertModel
from transformers import DistilBertForSequenceClassification

Now we can use the model's name from up above and build each piece ourselves. HuggingFace uses the `from_pretrained` method to make this quick and easy. 

In [101]:
# the model we are using
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'

In [102]:
# creating the config
config = DistilBertConfig.from_pretrained(model_name)

# creating the preprocessor 
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

# creating the model
model = DistilBertForSequenceClassification.from_pretrained(model_name)

Next we build a simple pipeline with these manual pieces.  

In [129]:
def preprocess(text):
    """
    Sends `text` through the LLM's tokenizer.  
    The tokenizers turns words and characters into special inputs for the LLM.
    """
    tokenized_inputs = tokenizer(text, return_tensors='pt')
    return tokenized_inputs


def forward(text):
    """
    First we preprocess the `text` into tokens.
    Then we send the `token_inputs` to the model.
    """
    token_inputs = preprocess(text)
    outputs = model(**token_inputs)
    return outputs

def process_outputs(outs):
    """
    Here is where HuggingFace does the most for us via `pipeline`.  

    """

    # grab the raw "scores" that from the model for Positive and Negative labels
    logits = outs.logits

    # find the strongest label score, aka the model's decision
    pred_idx = logits.argmax(1).item()

    # use the `config` object to find the class label
    pred_label = config.id2label[pred_idx]  

    # calculate the human-readable number for the score
    pred_score = logits.softmax(-1)[:, pred_idx].item()

    return {
        'label': pred_label,
        'score': pred_score, 
    }

def simple_pipeline(text):
    model_outs = forward(text)
    preds = process_outputs(model_outs)
    return preds

Let's call this pipeline on the same example text from before.

In [127]:
text = "We are very happy to show you the 🤗 Transformers library."

In [128]:
simple_pipeline(text)

{'label': 'POSITIVE', 'score': 0.9997795224189758}